## Set up the environment

In [18]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [19]:
import utils

utils.cifar10_download()

cifar dataset already downloaded


## Upload the dataset to an S3 bucket

In [20]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

In [21]:
!cat cifar10_cnn.py



from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import tensorflow as tf
from tensorflow.python.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.saved_model.signature_constants import PREDICT_INPUTS
from tensorflow.python.training.rmsprop import RMSPropOptimizer

HEIGHT = 32
WIDTH = 32
DEPTH = 3
NUM_CLASSES = 10
NUM_DATA_BATCHES = 5
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 10000 * NUM_DATA_BATCHES
BATCH_SIZE = 40
INPUT_TENSOR_NAME = 'inputs_input'  # needs to match the name of the first layer + "_input"


def keras_model_fn(hyperparameters):
    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', name='inputs', input_shape=(HEIGHT, WIDTH, DEPTH)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))

In [22]:
def keras_model_fn(hyperparameters):
    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', name='inputs', input_shape=(HEIGHT, WIDTH, DEPTH)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    opt = RMSPropOptimizer(learning_rate=hyperparameters['learning_rate'], decay=hyperparameters['decay'])

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [23]:
def serving_input_fn(params):
    # Notice that the input placeholder has the same input shape as the Keras model input
    tensor = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, DEPTH])
    
    # The inputs key INPUT_TENSOR_NAME matches the Keras InputLayer name
    inputs = {INPUT_TENSOR_NAME: tensor}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


def train_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.TRAIN,
                    batch_size=BATCH_SIZE, data_dir=training_dir)


def eval_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.EVAL,
                    batch_size=BATCH_SIZE, data_dir=training_dir)

In [24]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar10_cnn.py',
                       role=role,
                       framework_version='1.12.0',
                       hyperparameters={'learning_rate': 1e-4, 'decay':1e-6},
                       training_steps=100, evaluation_steps=20,
                       train_instance_count=1, train_instance_type='ml.c4.xlarge')

estimator.fit(inputs)

tensorflow py2 container will be deprecated soon.


2019-06-05 22:35:23 Starting - Starting the training job...
2019-06-05 22:35:25 Starting - Launching requested ML instances......
2019-06-05 22:36:27 Starting - Preparing the instances for training...
2019-06-05 22:37:09 Downloading - Downloading input data...
2019-06-05 22:37:50 Training - Training image download completed. Training in progress..
2019-06-05 22:37:44,613 INFO - root - running container entrypoint
2019-06-05 22:37:44,613 INFO - root - starting train task
2019-06-05 22:37:44,629 INFO - container_support.training - Training starting
2019-06-05 22:37:47,302 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2019-06-05 22:37:47,302 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2019-06-05 22:37:47,302 INFO - tf_container - ---------------------------------------------------------
2019-06-05 22:37:47,302 INFO - tf_container - creating RunConfig:
2019-06-05 22:37:47,30

In [25]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


--------------------------------------------------------------------------------------!

In [26]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)

# The inputs key 'inputs_input' matches the Keras InputLayer name
predictor.predict({'inputs_input': data}) 

{'outputs': {'activation_7': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 10}]},
   'float_val': [0.0962979644536972,
    0.10257883369922638,
    0.09764635562896729,
    0.09731990844011307,
    0.09547405689954758,
    0.09692122042179108,
    0.10776207596063614,
    0.10034969449043274,
    0.09994587302207947,
    0.10570395737886429]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1559774294},
  'signature_name': 'serving_default'}}

In [10]:
sagemaker.Session().delete_endpoint(predictor.endpoint)